In [2]:
import numpy as np
from pyscf import gto, scf, fci
from multiprocessing.pool import Pool
from numba import prange, jit, njit
from qmrdmft import Calculator, OptimalCalculator, tools
from tabulate import tabulate

#from numba.openmp import openmp_context as openmp

The purpose of this script is to caculate the electron electron interaction energy, or Levy Valone from FCI for
the first and second row of elements and to compare it to the approximations generated by various functionals, 
at the date of this writing, the Geodecker Umrigar, the Mueller, and BBC1-3. I test multiple ao basis sets, it is 
im-portant to note that only for the CCPVDZ and the 631G the whole set can be calculated. TZ can be done up to N, QZ can be done 
yup to Be.

Some Reference Data, Atom Symbols and Number of protons in PSE. Multiplicities of the groundstate 
in absence of charge in multiplicities, number of unpaired eelctrons coresponding to 
aforementoined multiplicities in spins.

In [3]:
PSE = {'H': 1,'He': 2,'Li': 3,'Be': 4,'B': 5,'C': 6,'N': 7,'O': 8,'F': 9,'Ne': 10,\
       'Na': 11,'Mg': 12,'Al': 13,'Si': 14,'P': 15,'S': 16,'Cl': 17,'Ar': 18,'K': 19,\
       'Ca': 20,'Sc': 21,'Ti': 22,'V': 23,'Cr': 24,'Mn': 25,'Fe': 26,'Co': 27,'Ni': 28,\
       'Cu': 29,'Zn': 30,'Ga': 31,'Ge': 32,'As': 33,'Se': 34,'Br': 35,'Kr': 36}
multiplicities = [2, 1, 2, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 4, 7, 6, 5, 4, 3, 2, 1, 2, 3, 4, 3, 2, 1]
spins = [ i-1 for i in multiplicities ]


In [4]:
def Add_Block_Matrices(Paa, Pbb):
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [5]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [6]:
# for jit and prange
@jit(nopython=True, parallel=True)
def ONERDMFT_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [7]:
# for spinorbitals
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in [b for b in range(0,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy
#                if not(a == b):


In [8]:
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in [b for b in range(0,M//2) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in [b for b in range(M//2,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    

    return -1/2.*energy

In [9]:
@jit(parallel=True)
def ONERDMFT_exchange_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [10]:
@jit(parallel=True)
def ONERDMFT_Mueller_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [11]:
@jit(parallel=True)
def ONERDMFT_BBC1(Fouridx, C, n, Na, Nb):
    energy = 0 
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(Na,M//2):
        for b in [b for b in range(Na,M//2) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])\
                            *C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]\
                            *Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2+Nb,M):
        for b in [b for b in range(M//2+Nb,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])\
                            *C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]\
                            *Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return energy

In [12]:
@jit(parallel=True)
def ONERDMFT_BBC2(Fouridx, C, n, Na, Nb):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,Na):
        for b in [b for b in range(0,Na) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += (np.sqrt(n[a]*n[b])-(n[a]*n[b]))\
                            *C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]\
                            *Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M//2+Nb):
        for b in [b for b in range(M//2,M//2+Nb) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += (np.sqrt(n[a]*n[b])-(n[a]*n[b]))\
                            *C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]\
                            *Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return  .5*energy

In [13]:
@jit(parallel=True)
def ONERDMFT_BBC3(Fouridx, C, n, Na, Nb):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for mu in range(0,M):
            for nu in range(0,M):
                for kappa in range(0,M):
                    for lamda in range(0,M):
                        energy += (np.sqrt(n[a]*n[a])-(n[a]*n[a]))\
                        *C[mu,a]*C[nu,a]*C[kappa,a]*C[lamda,a]\
                        *Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return energy

In [14]:
def energy_components_umrigar(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF):
    E_U = ONERDMFT_Umrigar_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    GU_E_xc = ONERDMFT_Umrigar_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    Vee = E_U + GU_E_xc
    E_tot = h1 + Vee
    E_c = E_tot - E_HF
    return E_tot, Vee, E_c 

In [15]:
def energy_components_mueller(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF):
    E_H = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    Mu_E_xc = ONERDMFT_Mueller_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    Vee = E_H + Mu_E_xc
    E_tot = h1 + Vee
    E_c = E_tot - E_HF
    return E_tot, Vee, E_c 

In [16]:
def energy_components_bbc1(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,nelec):
    n_a, n_b = nelec[0], nelec[1]
    E_H = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    Mu_E_xc = ONERDMFT_Mueller_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    BBC1 = ONERDMFT_BBC1(eri, FCInaturalCTTE, FCIoccuE,n_a,n_b)
    Vee = E_H + Mu_E_xc + BBC1
    E_tot = h1 + Vee + E_nn 
    E_c = E_tot - E_HF
    return E_tot, Vee, E_c 

In [17]:
def energy_components_bbc2(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,nelec):
    n_a, n_b = nelec[0], nelec[1]
    E_H = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    Mu_E_xc = ONERDMFT_Mueller_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    BBC1 = ONERDMFT_BBC1(eri, FCInaturalCTTE, FCIoccuE,n_a,n_b)
    BBC2 = ONERDMFT_BBC2(eri, FCInaturalCTTE, FCIoccuE,n_a,n_b)
    Vee = E_H + Mu_E_xc + BBC1 + BBC2
    E_tot = h1 + Vee + E_nn
    E_c = E_tot - E_HF
    return E_tot, Vee, E_c

In [18]:
def energy_components_bbc3(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,nelec):
    n_a, n_b = nelec[0], nelec[1]
    E_H = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    Mu_E_xc = ONERDMFT_Mueller_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
    BBC1 = ONERDMFT_BBC1(eri, FCInaturalCTTE, FCIoccuE,n_a,n_b)
    BBC2 = ONERDMFT_BBC2(eri, FCInaturalCTTE, FCIoccuE,n_a,n_b)
    BBC3 = ONERDMFT_BBC3(eri, FCInaturalCTTE, FCIoccuE,n_a,n_b)
    Vee = E_H + Mu_E_xc + BBC1 + BBC2 + BBC3
    E_tot = h1 + Vee + E_nn
    E_c = E_tot - E_HF
    return E_tot, Vee, E_c


In [21]:
stats = []
for el in list(PSE.keys())[0:10]:
# for el in ['Be']:
    mol = gto.Mole()
    mol.atom = f"""
        {el}    0.    0.    0.
    """
    # this basis has 2 functions for Helium
    mol.basis = "6-31g" 
    #mol.basis = "ccpvtz"
    # mol.basis = "sto-6g"
    mol.spin =  spins [PSE[el]-1 ] 
    mol.verbose=0
    mol.build()
    
    # the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
    eri = mol.intor('int2e')
    S = mol.intor('int1e_ovlp')
        
    ## Run Hartree-Fock.
    mf = scf.RHF(mol)
    mf.kernel()

    # Harvesting Fock Properties
    C = mf.mo_coeff
    h = mf.get_hcore()    
    N = mol.nelec[0]
    P=np.matmul(C[:,0:N],C[:,0:N].T)
    E_nn = mf.energy_nuc()


    # Translate Fock Properties into Fock Basis Set 
    #gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
    #occu, naturalC = np.linalg.eigh(gamma)
    
    #  get natural orb
    gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
    occu_aa, naturalC_aa = np.linalg.eigh(gamma)
    occu = np.append(occu_aa, occu_aa)

    # translate into AO basis and expand
    CE = expand_matrix(C)
    naturalCE = expand_matrix(naturalC_aa)
    naturalCTT = np.matmul(CE,naturalCE)
    for i, n  in enumerate(occu):
        if n < 0:
            occu[i] = 0 
            
# Run FCI            
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)

# Preparing Data for the Natural Orbital Functionals
    FCIgamma_a, FCIgamma_b = fci.direct_spin1.make_rdm1s(ci, mf.mo_coeff.shape[0], mol.nelec)
    FCIoccu_a, FCInaturalC_a = np.linalg.eigh(FCIgamma_a)
    FCIoccu_b, FCInaturalC_b = np.linalg.eigh(FCIgamma_b)
    FCInaturalC_a = FCInaturalC_a[:,::-1]
    FCInaturalC_b = FCInaturalC_b[:,::-1]
    FCIoccu_a = FCIoccu_a[::-1]
    FCIoccu_b = FCIoccu_b[::-1]
    FCInaturalCTT_a, FCInaturalCTT_b = np.matmul(C,FCInaturalC_a), np.matmul(C,FCInaturalC_b)
    FCInaturalCTTE = Add_Block_Matrices(FCInaturalCTT_a, FCInaturalCTT_b)
    FCIoccuE = np.append(FCIoccu_a,FCIoccu_b)
    for i, n  in enumerate(FCIoccuE):
        if n < 0:
            FCIoccuE[i] = 0

# Sorting Out FCI results
    E_HF = mf.e_tot
    FCI_tot = e
    FCI_c = e-E_HF
    h1 = np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma_a,C.T)))) + np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma_b,C.T))))
    FCI_Vee = FCI_tot - h1

# calling 1RDMFT energy functions
    GU_tot,GU_Vee,GU_E_c = energy_components_umrigar(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF)
    Mu_tot,Mu_Vee,Mu_E_c = energy_components_mueller(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF)
    BBC1_tot, BBC1_Vee, BBC1_E_c = energy_components_bbc1(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,mol.nelec)
    BBC2_tot, BBC2_Vee, BBC2_E_c = energy_components_bbc2(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,mol.nelec)
    BBC3_tot, BBC3_Vee, BBC3_E_c = energy_components_bbc3(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,mol.nelec)

    stats.append([el, GU_E_c, Mu_E_c, BBC1_E_c, BBC2_E_c, BBC3_E_c, FCI_c, GU_tot, Mu_tot, BBC1_tot, BBC2_tot, BBC3_tot,FCI_tot, GU_Vee, Mu_Vee, BBC1_Vee, BBC2_Vee, BBC3_Vee, FCI_Vee]) 

In [22]:
top = "El. Ec_{GU} Ec_{Mu} Ec_{BBC1} Ec_{BBC2} Ec_{BBC3} Ec_{FCI}"
np_stats=np.asarray(stats)
print("\nCorrelation Energies")
print(tabulate(np_stats[:,0:7],headers=top.split(), tablefmt="latex" ) )

top = "El. Etot_{GU} Etot_{Mu} Etot_{BBC1} Etot_{BBC2} Etot_{BBC3} Etot_{FCI}"
print("\nTotal Energies")
print(tabulate(np.c_[np_stats[:,0], np_stats[:,7:13]]  ,headers=top.split(), tablefmt="latex")  )

top = "El. Vee_{GU} Vee_{Mu} Vee_{BBC1} Vee_{BBC2} Vee_{BBC3} Vee_{FCI}"
print("\nElectron Electron Interaction Energies")
print(tabulate(np.c_[np_stats[:,0], np_stats[:,13:]]  ,headers=top.split(), tablefmt="latex")  )


Correlation Energies
\begin{tabular}{lrrrrrr}
\hline
 El.   &      Ec\_\{GU\} &      Ec\_\{Mu\} &    Ec\_\{BBC1\} &    Ec\_\{BBC2\} &    Ec\_\{BBC3\} &     Ec\_\{FCI\} \\
\hline
 H     & -1.66533e-16 & -1.66533e-16 & -1.66533e-16 & -1.66533e-16 & -1.66533e-16 & -1.11022e-16 \\
 He    & -0.00793891  & -0.0156566   & -0.0156566   & -0.0156566   & -0.000221229 & -0.0150017   \\
 Li    & -0.00110736  & -0.00124189  & -0.0012293   & -0.00122317  & -0.000954112 & -0.000319235 \\
 Be    & -0.00609827  & -0.0758716   & -0.0652868   & -0.0629278   &  0.0766188   & -0.0467812   \\
 B     & -0.0296542   & -0.0853991   & -0.0771239   & -0.0723694   &  0.0391205   & -0.0435438   \\
 C     & -0.0453417   & -0.0820681   & -0.076527    & -0.0716685   &  0.00178411  & -0.0393988   \\
 N     & -0.0528698   & -0.0704851   & -0.0660436   & -0.0630412   & -0.0278105   & -0.0378885   \\
 O     & -0.0663242   & -0.092907    & -0.0867407   & -0.0815957   & -0.02843     & -0.0611739   \\
 F     & -0.0791366  

In [20]:
top = "El. Vee_{GU} Vee_{Mu} Vee_{BBC1} Vee_{BBC2} Vee_{BBC3} Vee_{FCI}"
print("\nElectron Electron Interaction Energies")
np_stats_rel = np.asarray(np_stats[:,13:],dtype=float)
for column in range(0,np_stats_rel.shape[1]):
    print(np_stats_rel[:,column], np_stats_rel[:,-1])
    np_stats_rel[:,column] = np_stats_rel[:,column] - np_stats_rel[:,-1]
    np_stats_rel[:,column] = np_stats_rel[:,column]*100
print(tabulate(np.c_[np_stats[:,0], np_stats_rel[:,0:-3]]  ,headers=top.split(), tablefmt="latex")  )


Electron Electron Interaction Energies
[4.54516457] [4.5044816]
[4.47539128] [4.5044816]
[4.48597606] [4.5044816]
[4.48833501] [4.5044816]
[4.62788161] [4.5044816]
[4.5044816] [4.5044816]
\begin{tabular}{lrrr}
\hline
 El.   &   Vee\_\{GU\} &   Vee\_\{Mu\} &   Vee\_\{BBC1\} \\
\hline
 Be    &     4.0683 &   -2.90903 &     -1.85055 \\
\hline
\end{tabular}


In [21]:
stats = []
for el in list(PSE.keys())[0:10]:
    mol = gto.Mole()
    mol.atom = f"""
        {el}    0.    0.    0.
    """
    # this basis has 2 functions for Helium
    mol.basis = "ccpvdz"
    #mol.basis = "sto-6g"
    mol.spin =  spins [PSE[el]-1 ] 
    mol.verbose=0
    mol.build()
    
    # the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
    eri = mol.intor('int2e')
    S = mol.intor('int1e_ovlp')
        
    ## Run Hartree-Fock.
    mf = scf.RHF(mol)
    mf.kernel()

    # Harvesting Fock Properties
    C = mf.mo_coeff
    h = mf.get_hcore()    
    N = mol.nelec[0]
    P=np.matmul(C[:,0:N],C[:,0:N].T)
    E_nn = mf.energy_nuc()


    # Translate Fock Properties into Fock Basis Set 
    #gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
    #occu, naturalC = np.linalg.eigh(gamma)
    
    #  get natural orb
    gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
    occu_aa, naturalC_aa = np.linalg.eigh(gamma)
    occu = np.append(occu_aa, occu_aa)

    # translate into AO basis and expand
    CE = expand_matrix(C)
    naturalCE = expand_matrix(naturalC_aa)
    naturalCTT = np.matmul(CE,naturalCE)
    for i, n  in enumerate(occu):
        if n < 0:
            occu[i] = 0 
            
# Run FCI            
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)

# Preparing Data for the Natural Orbital Functionals
    FCIgamma_a, FCIgamma_b = fci.direct_spin1.make_rdm1s(ci, mf.mo_coeff.shape[0], mol.nelec)
    FCIoccu_a, FCInaturalC_a = np.linalg.eigh(FCIgamma_a)
    FCIoccu_b, FCInaturalC_b = np.linalg.eigh(FCIgamma_b)
    FCInaturalC_a = FCInaturalC_a[:,::-1]
    FCInaturalC_b = FCInaturalC_b[:,::-1]
    FCIoccu_a = FCIoccu_a[::-1]
    FCIoccu_b = FCIoccu_b[::-1]
    FCInaturalCTT_a, FCInaturalCTT_b = np.matmul(C,FCInaturalC_a), np.matmul(C,FCInaturalC_b)
    FCInaturalCTTE = Add_Block_Matrices(FCInaturalCTT_a, FCInaturalCTT_b)
    FCIoccuE = np.append(FCIoccu_a,FCIoccu_b)
    for i, n  in enumerate(FCIoccuE):
        if n < 0:
            FCIoccuE[i] = 0

# Sorting Out FCI results
    E_HF = mf.e_tot
    FCI_tot = e
    FCI_c = e-E_HF
    h1 = np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma_a,C.T)))) + np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma_b,C.T))))
    FCI_Vee = FCI_tot - h1

# calling 1RDMFT energy functions
    GU_tot,GU_Vee,GU_E_c = energy_components_umrigar(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF)
    Mu_tot,Mu_Vee,Mu_E_c = energy_components_mueller(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF)
    BBC1_tot, BBC1_Vee, BBC1_E_c = energy_components_bbc1(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,mol.nelec)
    BBC2_tot, BBC2_Vee, BBC2_E_c = energy_components_bbc2(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,mol.nelec)
    BBC3_tot, BBC3_Vee, BBC3_E_c = energy_components_bbc3(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,mol.nelec)

    stats.append([el, GU_E_c, Mu_E_c, BBC1_E_c, BBC2_E_c, BBC3_E_c, FCI_c, GU_tot, Mu_tot, BBC1_tot, BBC2_tot, BBC3_tot,FCI_tot, GU_Vee, Mu_Vee, BBC1_Vee, BBC2_Vee, BBC3_Vee, FCI_Vee])

In [22]:
top = "El. Ec_{GU} Ec_{Mu} Ec_{BBC1} Ec_{BBC2} Ec_{BBC3} Ec_{FCI}"
np_stats=np.asarray(stats)
print("Correlation Energies")
print(tabulate(np_stats[:,0:7],headers=top.split())  )

top = "El. Etot_{GU} Etot_{Mu} Etot_{BBC1} Etot_{BBC2} Etot_{BBC3} Etot_{FCI}"
print("\nTotal Energies")
print(tabulate(np.c_[np_stats[:,0], np_stats[:,7:13]]  ,headers=top.split())  )

top = "El. Vee_{GU} Vee_{Mu} Vee_{BBC1} Vee_{BBC2} Vee_{BBC3} Vee_{FCI}"
print("\nElectron Electron Interaction Energies")
print(tabulate(np.c_[np_stats[:,0], np_stats[:,13:]]  ,headers=top.split())  )

Correlation Energies
El.         Ec_{GU}       Ec_{Mu}     Ec_{BBC1}     Ec_{BBC2}     Ec_{BBC3}      Ec_{FCI}
-----  ------------  ------------  ------------  ------------  ------------  ------------
H      -1.11022e-16  -1.11022e-16  -1.11022e-16  -1.11022e-16  -1.11022e-16  -1.66533e-16
He     -0.0216852    -0.0354631    -0.0339652    -0.0339652    -0.00640934   -0.0324344
Li     -0.000936922  -0.00103268   -0.00101985   -0.00101711   -0.000825589  -0.000217635
Be     -0.0121118    -0.0759938    -0.0627981    -0.0608683     0.0668957    -0.0450719
B      -0.0454129    -0.109545     -0.0896179    -0.0843339     0.0439304    -0.0640387
C      -0.0764119    -0.131962     -0.112294     -0.105688      0.00541285   -0.0794868
N      -0.101655     -0.14229      -0.128063     -0.122581     -0.0413101    -0.0917008
O      -0.126295     -0.171627     -0.156151     -0.148229     -0.0575668    -0.124231
F      -0.147467     -0.196538     -0.180323     -0.170586     -0.0724443    -0.157656
Ne   

In [23]:
stats = []
for el in list(PSE.keys())[0:6]:
    mol = gto.Mole()
    mol.atom = f"""
        {el}    0.    0.    0.
    """
    # this basis has 2 functions for Helium
    mol.basis = "ccpvtz"
    #mol.basis = "sto-6g"
    mol.spin =  spins [PSE[el]-1 ] 
    mol.verbose=0
    mol.build()
    
    # the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
    eri = mol.intor('int2e')
    S = mol.intor('int1e_ovlp')
        
    ## Run Hartree-Fock.
    mf = scf.RHF(mol)
    mf.kernel()

    # Harvesting Fock Properties
    C = mf.mo_coeff
    h = mf.get_hcore()    
    N = mol.nelec[0]
    P=np.matmul(C[:,0:N],C[:,0:N].T)
    E_nn = mf.energy_nuc()


    # Translate Fock Properties into Fock Basis Set 
    #gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
    #occu, naturalC = np.linalg.eigh(gamma)
    
    #  get natural orb
    gamma = np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
    occu_aa, naturalC_aa = np.linalg.eigh(gamma)
    occu = np.append(occu_aa, occu_aa)

    # translate into AO basis and expand
    CE = expand_matrix(C)
    naturalCE = expand_matrix(naturalC_aa)
    naturalCTT = np.matmul(CE,naturalCE)
    for i, n  in enumerate(occu):
        if n < 0:
            occu[i] = 0 
            
# Run FCI            
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)

# Preparing Data for the Natural Orbital Functionals
    FCIgamma_a, FCIgamma_b = fci.direct_spin1.make_rdm1s(ci, mf.mo_coeff.shape[0], mol.nelec)
    FCIoccu_a, FCInaturalC_a = np.linalg.eigh(FCIgamma_a)
    FCIoccu_b, FCInaturalC_b = np.linalg.eigh(FCIgamma_b)
    FCInaturalC_a = FCInaturalC_a[:,::-1]
    FCInaturalC_b = FCInaturalC_b[:,::-1]
    FCIoccu_a = FCIoccu_a[::-1]
    FCIoccu_b = FCIoccu_b[::-1]
    FCInaturalCTT_a, FCInaturalCTT_b = np.matmul(C,FCInaturalC_a), np.matmul(C,FCInaturalC_b)
    FCInaturalCTTE = Add_Block_Matrices(FCInaturalCTT_a, FCInaturalCTT_b)
    FCIoccuE = np.append(FCIoccu_a,FCIoccu_b)
    for i, n  in enumerate(FCIoccuE):
        if n < 0:
            FCIoccuE[i] = 0

# Sorting Out FCI results
    E_HF = mf.e_tot
    FCI_tot = e
    FCI_c = e-E_HF
    h1 = np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma_a,C.T)))) + np.trace(np.matmul(h,np.matmul(C, np.matmul(FCIgamma_b,C.T))))
    FCI_Vee = FCI_tot - h1

# calling 1RDMFT energy functions
    GU_tot,GU_Vee,GU_E_c = energy_components_umrigar(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF)
    Mu_tot,Mu_Vee,Mu_E_c = energy_components_mueller(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF)
    BBC1_tot, BBC1_Vee, BBC1_E_c = energy_components_bbc1(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,mol.nelec)
    BBC2_tot, BBC2_Vee, BBC2_E_c = energy_components_bbc2(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,mol.nelec)
    BBC3_tot, BBC3_Vee, BBC3_E_c = energy_components_bbc3(eri, FCInaturalCTTE, FCIoccuE,h1,E_HF,E_nn,mol.nelec)

    stats.append([el, GU_E_c, Mu_E_c, BBC1_E_c, BBC2_E_c, BBC3_E_c, FCI_c, GU_tot, Mu_tot, BBC1_tot, BBC2_tot, BBC3_tot,FCI_tot, GU_Vee, Mu_Vee, BBC1_Vee, BBC2_Vee, BBC3_Vee, FCI_Vee])

KeyboardInterrupt: 

In [ ]:
top = "El. Ec_{GU} Ec_{Mu} Ec_{BBC1} Ec_{BBC2} Ec_{BBC3} Ec_{FCI}"
np_stats=np.asarray(stats)
print("Correlation Energies")
print(tabulate(np_stats[:,0:7],headers=top.split())  )

top = "El. Etot_{GU} Etot_{Mu} Etot_{BBC1} Etot_{BBC2} Etot_{BBC3} Etot_{FCI}"
print("\nTotal Energies")
print(tabulate(np.c_[np_stats[:,0], np_stats[:,7:13]]  ,headers=top.split())  )

top = "El. Vee_{GU} Vee_{Mu} Vee_{BBC1} Vee_{BBC2} Vee_{BBC3} Vee_{FCI}"
print("\nElectron Electron Interaction Energies")
print(tabulate(np.c_[np_stats[:,0], np_stats[:,13:]]  ,headers=top.split())  )